In [ ]:
!pip install hazm

In [1]:
import logging
from gensim.models import Word2Vec
from hazm import Normalizer, word_tokenize

#فعالسازی لاگینگ برای نمایش پیشرفت اموزش مدل
logging.basicConfig(format='%(asctime)s: %(levelname)s : %(message)s',level=logging.INFO)

#لیست جملات
documents=["ببر و شیر از بزرگترین گربه‌سانان هستند",
           "شیر به خاطر بدن بزرگ و هیبت و چالاکیش در هنگام شکار لقب <<سلطان جنگل>> را به خود گرفته اگرچه بیشتر در مناطق گرم دشت افریقا زندگی می‌کمد",
           "دو توله شیر سفید افریقایی که سال 98 وارد کشور شدند. اکنون حدود 9 ماه دارند و در باغ وحش ارم نگهداری می شود",
           "نوشیدن روزانه ی شیر فواید زیادی برای سلامتی دارد"]


#نرمال ساز
normalizer=Normalizer()

#پیش پردازش و توکن سازی جملات
normalized_doc= [normalizer.normalize(doc) for doc in documents]
tokenized_doc= [word_tokenize(doc) for doc in normalized_doc]

#اموزش مدل word2vec
model= Word2Vec(sentences= tokenized_doc,vector_size=100,window=5,min_count=1,workers=4)
      #جملات    1
      #سایز بردار بازنمایی متراکم (هرچه بیشتر باشه فیچر بیشتر استخراج میکنه که همیشه باید ی چیز بینابین باشه)  2
      #3  تعداد پنجره
      #4,5  مربوط به بحث پرسسوری


#نمایش بردار امبدینگ کلمه شیر
print("بردار کلمه شیر")
print(model.wv['شیر'])

#ذخیره مدل اموزش دیده
model.save("persian_word2vec.model")

بردار کلمه شیر
[-8.6244084e-03  3.6660081e-03  5.1921923e-03  5.7491604e-03
  7.4761165e-03 -6.1893831e-03  1.1150864e-03  6.0711624e-03
 -2.8437865e-03 -6.1892429e-03 -4.0249387e-04 -8.3879903e-03
 -5.6088739e-03  7.1173473e-03  3.3572393e-03  7.2285761e-03
  6.8025952e-03  7.5335889e-03 -3.7942315e-03 -5.7509396e-04
  2.3480570e-03 -4.5233201e-03  8.3964961e-03 -9.8732170e-03
  6.7682625e-03  2.9031064e-03 -4.9425317e-03  4.3975418e-03
 -1.7440682e-03  6.7079994e-03  9.9736061e-03 -4.3646484e-03
 -5.8340712e-04 -5.7097310e-03  3.8500256e-03  2.7889416e-03
  6.8989852e-03  6.1071478e-03  9.5442412e-03  9.2661334e-03
  7.8984033e-03 -6.9941562e-03 -9.1712307e-03 -3.5225795e-04
 -3.0996727e-03  7.8953011e-03  5.9363469e-03 -1.5373618e-03
  1.5137295e-03  1.8001864e-03  7.8193871e-03 -9.5148161e-03
 -1.9696944e-04  3.4738528e-03 -9.2895713e-04  8.3880806e-03
  9.0261018e-03  6.5329610e-03 -7.0929690e-04  7.7289278e-03
 -8.5417675e-03  3.2033802e-03 -4.6394705e-03 -5.0943573e-03
  3.58175

In [2]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296189 sha256=bb723e0d9a2f1afb4fde4983c3cb2bea9c94919f608db2a7dcc29084b4525336
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


خب اینجا میخاییم روی  fasttext انجام بدیم

فست تکست متن بازه

In [ ]:
import fasttext
import fasttext.util
from hazm import Normalizer
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

#download the persian fasttext model
fasttext.util.download_model('fa',if_exists="ignore")


#load the model
model= fasttext.load_model('cc.fa.300.bin')

#initialize persian text normalizer
normalizer= Normalizer()

def get_word_vector(word):
  normalized_word=normalizer.normalize(word)
  return model.get_word_vector(normalized_word)


def visualize_word_embeddings(words):
  #get word vectors
  word_vectors= np.array([get_word_vector(word) for word in words])

  #redius dimensionality to 2D useing PCA
  #روش پی سی ای برای دو بعدی کردن طبیعتا یه بخشی از داده از بین میره
  pca=PCA(n_components=2)
  word_vector_2d= pca.fit_transform(word_vectors)

  #plot the words
  plt.figure(figesize=(12,8))
  plt.scatter(word_vector_2d[:,0],word_vector_2d[:,1],marker="o")

  #add word labels to the plot
  for i,word in enumerate(words):
    plt.annotate(word,(word_vector_2d[i,0], word_vector_2d[i,1]),fontsize=0)

  plt.titel("Persian Word Embeddings Visualization")
  plt.xlabel("First principal Component")
  plt.ylabel("Second principal Component")
  plt.grid(True)


  #add text explanation     (بحث زیباییه زیاد مهم نی)
  plt.text(0.05,0.05,"Words that are closer together in this plot have more similar meanings according to the fasttext model.",
           transform=plt.gca.transAxes,fontsize=9,verticalalignment='bottom')
  plt.tight_layout()
  plt.show()


#exampel usage with a mix of word
persian_words=["کتاب",
               "مجله",
               "روزنامه",
               "قلم",
               "کاغذ",
               "مدرسه",
               "دانشگاه",
               "دانش",
               "ماشین",
               "اتوبوس",
               "دوچرخه",
               "غذا",
               "اب",
               "هوا",
               "زمین"]

visualize_word_embeddings(persian_words)

In [2]:
import logging
from gensim.models import FastText
from hazm import Normalizer, word_tokenize

#فعالسازی لاگینگ برای نمایش پیشرفت اموزش مدل
logging.basicConfig(format='%(asctime)s: %(levelname)s : %(message)s',level=logging.INFO)

#لیست جملات
documents=["ببر و شیر از بزرگترین گربه‌سانان هستند",
           "شیر به خاطر بدن بزرگ و هیبت و چالاکیش در هنگام شکار لقب <<سلطان جنگل>> را به خود گرفته اگرچه بیشتر در مناطق گرم دشت افریقا زندگی می‌کمد",
           "دو توله شیر سفید افریقایی که سال 98 وارد کشور شدند. اکنون حدود 9 ماه دارند و در باغ وحش ارم نگهداری می شود",
           "نوشیدن روزانه ی شیر فواید زیادی برای سلامتی دارد"]


#نرمال ساز
normalizer=Normalizer()

#پیش پردازش و توکن سازی جملات
normalized_doc= [normalizer.normalize(doc) for doc in documents]
tokenized_doc= [word_tokenize(doc) for doc in normalized_doc]

#اموزش مدل fasttext
model= FastText(sentences= tokenized_doc,vector_size=100,window=5,min_count=1,workers=4)



#نمایش بردار امبدینگ کلمه شیر
print("بردار کلمه شیر")
print(model.wv['شیر'])

#ذخیره مدل اموزش دیده
model.save("persian_FastText.model")

بردار کلمه شیر
[-1.2556983e-03 -3.6090877e-04  1.3299880e-03 -1.5113958e-03
  1.4843381e-03  3.5023228e-03 -8.7301909e-05  2.5393909e-03
 -5.5496679e-03  2.5543382e-03 -1.1613303e-03 -3.5655312e-04
 -3.6893522e-03  4.1935015e-03  8.2855060e-04 -4.4681571e-04
 -2.7262853e-03  3.8359459e-03  3.1601917e-03 -3.2531689e-03
  3.0154185e-04 -5.8776687e-04  2.3710968e-04  2.4198215e-04
 -5.9797999e-04  1.9313399e-03 -8.4666407e-04  2.9676328e-03
 -5.1586293e-03 -3.4705300e-03 -1.0675795e-03 -7.2569586e-04
  2.2440800e-03 -7.9428649e-04  9.8960765e-04  2.4658017e-04
 -3.7067770e-05 -7.0743059e-04  3.0906638e-04  2.8088265e-03
  3.4439245e-03 -3.9901919e-04  2.6888826e-03 -1.6706161e-03
 -2.7205688e-03 -3.5617079e-03  1.2839666e-04 -1.9479344e-04
 -3.6495313e-04 -1.6650102e-03  4.2313179e-03 -2.8111201e-04
 -1.6570360e-03  4.0193181e-04  7.2608102e-04 -2.3523581e-03
  8.9927111e-04  4.9513200e-04 -2.3203196e-03 -8.6829870e-04
 -1.7223824e-03 -5.2395568e-04 -3.1575356e-03  1.8616825e-03
 -1.26300